# Bonus : sélectionner un document dans la DataTable et l'afficher avec IIIF

Bonjour 👋 !

Bienvenue dans cette partie bonus dédiée à l'ajout d'un **visualiseur IIIF** permettant d'afficher le document numérisé d'une édition de presse à partir de la table de données du ***dashboard* `dataset.py`**.

# A/ Ajout du visualiseur IIIF Mirador

L'objectif est simple : ajouter dans le *dashboard* un nouveau composant destiné à stocker une **IFrame HTML** offrant une vue vers la démo en ligne du visualiseur IIIF Mirador  : [https://projectmirador.org/embed/](https://projectmirador.org/embed/).

Les modifications sont à apporter à `dataset.py`.

Ajoutez au conteneur principal (le composant `Div`) un nouveau composant `dash.html.IFrame()`et :
- identifiez-le : `id="mirador"`
- spécifiez la taille de l'IFrame : `width="100%"` (100% de la largeur de la fenêtre), et `height="800px"` (la hauteur doit être une valeur absolue).


L'Iframe sera ajoutée sous la table de données, qui malheureusement peut être très longue. Pour la forcer dans des dimensions fixes, passez au composant `DataTable` le paramètrage suivant : 
```python
    style_table={
                "maxHeight": "400px", # La table mesurera au plus 400 pixels de haut.
                "overflowY": "scroll",      # Si la table dépasse, elle deviendra défilable.
            }
```

# B/ Permettre la sélection d'une ligne dans la `DataTable`

Pour rajouter la possibilité de sélectionner une ligne, on utilise un peu de "magie" Dash en ajoutant tout simplement au composant `DataTable`le paramètre `row_selectable="single"`.


Cela nous donne la possibilité de créer une nouvelle fonction *callback* qui sera déclenchée **lorsqu'une ligne est sélectionnée** et qui renverra le contenu à afficher dans l'Iframe.


Définissez une nouvelle fonction `def affiche_iiif_documents(idx_lignes_sélectionnées, table)` où :
- `idx_lignes_sélectionnées` sera la liste des numéros de lignes sélectionnées. Comme on a spécifié qu'une seule ligne était sélectionnable à la fois (avec `row_selectable="single"`), cette liste contiendra toujours un seul numéro ;
- `table` est le contenu de la `DataTable`, c'est à dire une liste de dictionnaire ;


Créez le corps de cette fonction pour qu'elle :
1. récupère la colonne `"ark"` de la ligne sélectionnée ;
2. construise l'URL attendue par l'API IIIF de Gallica afin de récupérer le fichier `manifest.json` correspondant à cet identifiat ARK ;
3. compose l'URL finale de la page à inclure dans l'IFrame :  `"https://projectmirador.org/embed/?iiif-content=URL_API"` où `URL_API` est l'URL crée au point 2.
4. renvoie cette URL

<span style="color: #40d6d1"><strong>💡 Astuce</strong></span> Pour composer facilement des chaînes de caractères à partir de variables, utilisez **l'interpolation de chaînes de caractères en Python** avec la syntaxe dite **f-strings** : 
C'est à dire :

```python
ark = "bpt6k451348m"

gallica_iiif = f"https://gallica.bnf.fr/iiif/ark:/12148/{ark}/manifest.json" # Noter le préfix f devant la chaîne de caractère, qui déclare à l'interpréteur Python qu'il doit remplacer les passages entre accolades par la valeur de la variable correspondante : ici {ark}.

iframe_embed = f"https://projectmirador.org/embed/?iiif-content={gallica_iiif}" # Encore une interposlation f-string :)
```

Reste enfin à déclarer le callback, comme dans le notebook principal !

Ici, on veut:
- en **Input** : les éléments  `selected_rows"=` et `"=data` venus du composant `DataTable` (identifiant `"table-corpus"`)
- en**Output**, vers le composant `IFrame`(identifiant `"mirador"`), l'élément `src`.

Au total : 
```python

@dash.callback(
    dash.Output("mirador", "src"),
    [dash.Input("table-corpus", "selected_rows")],
    [dash.Input("table-corpus", "data")],
)
def affiche_iiif_documents(idx_lignes_sélectionnées, table):
    ...
```

Rajoutez cet entête au dessus de la fonction `affiche_iiif_documents()` et testez votre application !
